# Sistemi preporuka zasnovani na uzajamnom filtriranju koristeci matricu kovarijansi

In [1]:
import pandas as pd
import numpy as np

In [5]:
header = ['user', 'item', 'rating', 'timestemp']

In [6]:
data = pd.read_table('data/ratings.dat', sep = '::', names = header)

/home/vladana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [7]:
data.head()

,user,item,rating,timestemp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user       1000209 non-null  int64
 1   item       1000209 non-null  int64
 2   rating     1000209 non-null  int64
 3   timestemp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [14]:
number_of_users = data['user'].unique().shape[0]

In [15]:
number_of_users

6040

In [17]:
number_of_items = data['item'].unique().shape[0]

In [27]:
number_of_items

3706

In [28]:
max_item = data['item'].max()

In [29]:
data['item'].min()

1

In [30]:
data['user'].min()

1

In [31]:
data['user'].max()

6040

In [32]:
user_item_matrix = np.zeros((number_of_users, max_item))
for row in data.itertuples():
    user_item_matrix[row[1]-1, row[2]-1] = row[3]

In [34]:
user_item_matrix

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 0.]])

In [35]:
zero = np.sum(user_item_matrix == 0)

In [37]:
zero

22869871

In [38]:
user_item_matrix.shape

(6040, 3952)

In [40]:
fields = user_item_matrix.shape[0]*user_item_matrix.shape[1]

In [42]:
fields

23870080

In [41]:
fields - zero

1000209

In [50]:
common_items = (user_item_matrix[0,:] != 0) & (user_item_matrix[1,:] !=0)

In [56]:
np.sum(common_items==True)

7

In [53]:
user_item_matrix[0, common_items]

array([5., 4., 4., 4., 5., 3., 5.])

In [59]:
def calculate_covariance_between_users(ui_matrix, i, j):
    if (i != j):
        ratings_i = ui_matrix[i,:]
        ratings_j = ui_matrix[j,:]
    
        common_items = (ratings_i != 0) & (ratings_j != 0)
    
        part1 = np.sum(ratings_i[common_items]*ratings_j[common_items])*(1/number_of_items)
        part2 = np.sum(ratings_i)*np.sum(ratings_j)*(1/(number_of_items**2))
    
        return part1 - part2
    else:
        return float('inf')
    

In [71]:
calculate_covariance_between_users(user_item_matrix, 1, 2)

0.04432792143776417

In [72]:
calculate_covariance_between_users(user_item_matrix, 1, 1)

inf

In [73]:
covariance_matrix = np.zeros((number_of_users, max_item))

In [74]:
for i in range(number_of_users):
    for j in range(i, max_item):
        covariance_matrix[i, j] = calculate_covariance_between_users(user_item_matrix, i, j)
        covariance_matrix[j, i] = covariance_matrix[i, j]

In [75]:
covariance_matrix

array([[       inf, 0.02733581, 0.02565551, ..., 0.01383763, 0.06038005,
        0.02899675],
       [0.02733581,        inf, 0.04432792, ..., 0.02034587, 0.11302721,
        0.02260916],
       [0.02565551, 0.04432792,        inf, ..., 0.01799877, 0.10041657,
        0.00967743],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [76]:
covariance_matrix[1,2]

0.04432792143776417

In [77]:
covariance_matrix[2, 1]

0.04432792143776417

In [78]:
covariance_matrix[0,0]

inf

In [79]:
covariance_matrix[6039, 0]

0.0